In order to use all of this though, setup a Developer API account with Twitter and create an application to get credentials.

In [0]:
!pip install Tweepy

     |████████████████████████████████| 101 kB 832 kB/s 
     |████████████████████████████████| 151 kB 5.6 MB/s 
     |████████████████████████████████| 62 kB 1.7 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


## Develop the TweetRead Program

In [0]:
import tweepy
import io
import json
import time
import os


# todo: Set up your credentials
consumer_key=''
consumer_secret=''
access_token =''
access_token_secret=''

class TweetsListener(tweepy.Stream):
  counter = 0 # for data counter.
  tweets_per_file = 10 
  buffer = []
  directory = None

  # on_data is an event that gets triggered each time there is new data (tweet) coming in.
  def on_data(self, data):
      try:
          
         
          msg = json.loads(data)
          tweet = {"time":time.strftime('%Y-%m-%dT%H:%M:%SZ', time.strptime(msg['create_at'],'%a %b %d %H:%M:%S +0000 %Y')), "text":msg['text']}
          
          print(f"{tweet['time']}- {tweet['text']}")
          self.buffer.append(tweet)
          self.counter= self.counter+1

          if(self.counter >= self.tweets_per_file):
           
            try:
              #create a file using the timestamp time.strftime("%Y%m%d-%H%M%S"), dump buffer into the file, then reset buffer
              timestr = time.strftime("%Y%m%d-%H%M%S")
              with io.open(self.directory+"/"+timestr +".txt", "w", encode = "utf8") as f:
                for row in self.buff:
                  f.write(json.dump(row))
                  f.write("\n")
                
                f.colse()
                self.counter = 0
                self.buffer = []
              
              
            except BaseException as e:
              print("error opening file:%s" % str(e))
          return True
      except BaseException as e:
          # if there is any error in processing the data, we print it on screen.
          print("Error on_data: %s" % str(e))
      return True
  
  # on_error gets triggered if there is some sort of error.
  def on_error(self, status):
      
      print(status)
      return True

def sendData(directory):  
  
  twitter_stream = TweetsListener(consumer_key,  consumer_secret, access_token, access_token_secret)
  twitter_stream.directory = directory
  twitter_stream.filter(track=['covid'], languages=['en'])
  
try:
  os.mkdir("/databricks/driver/tweets")
except Exception:
  # if the dir already exists
  pass

sendData("/databricks/driver/tweets")


In [0]:
import tweepy

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

public_tweets = api.home_timeline()
for tweet in public_tweets:
    print(tweet.text)